### Import libraries

In [ ]:
import os
import getpass
import proactive

### Creating the gateway

In [ ]:
print("Logging on proactive-server 1...")
proactive_host = 'try.activeeon.com'
proactive_port = '8080'
proactive_url  = "http://"+proactive_host+":"+proactive_port
print("Creating gateway ")
gateway = proactive.ProActiveGateway(proactive_url, debug=False)
print("Gateway created")

### Connecting

In [ ]:
print("Connecting on: " + proactive_url)
gateway.connect(username="YOUR_LOGIN", password="YOUR_PASSWORD")  # put your login here!
# Or uncomment the following line to protect your password
# gateway.connect(username="", password=getpass.getpass(prompt='Password: '))
assert gateway.isConnected() is True
print("Connected")

### Creating the _init_ task

In [ ]:
print("Creating the initialization task...")
init_task = gateway.createPythonTask()
init_task.setTaskName("init")
init_task.setTaskImplementation("""import os
pid = os.getpid()
print('Process ' + str(pid) + ': Initialization phase')
a = 2
b = 3
variables.put("a", a)
variables.put("b", b)""")
print("Task created.")

### Creating _calculation_ task

In [ ]:
print("Creating the initialization task...")
calculation_task = gateway.createPythonTask()
calculation_task.setTaskName("calculation")
calculation_task.setTaskImplementation("""import os
a = variables.get("a")
b = variables.get("b")
pid = os.getpid()
print('Process ' + str(pid) + ': Calculation phase')
print('Recieved a = ' + str(a))
print('Recieved b = ' + str(b))
c = a + b
print('Sum = ' + str(c))""")
calculation_task.addDependency(init_task)
print("Task created.")

### Creating the workflow

In [ ]:
print("Creating a proactive job...")
proactive_job = gateway.createJob()
proactive_job.setJobName("PythonPidWorkflow")
print("Job created.")

### Adding tasks to the workflow

In [ ]:
print("Adding tasks to the job...")
proactive_job.addTask(init_task)
proactive_job.addTask(calculation_task)
print("Tasks added.")

### Submitting the job to the scheduler

In [ ]:
print("Submitting the job to the proactive scheduler...")
job_id = gateway.submitJob(proactive_job, debug=False)
print("job_id: " + str(job_id))

### Getting job results and outputs

In [ ]:
print("Getting job results...")
job_result = gateway.getJobResult(job_id)
print(job_result)
print("Getting job outputs...")
job_outputs = gateway.printJobOutput(job_id)
print(job_outputs)

### Disconnecting

In [ ]:
print("Disconnecting")
gateway.disconnect()
print("Disconnected")
gateway.terminate()
print("Finished")